In [2]:
from keras.models import  Model
import tensorflow as tf
import numpy as np
import cv2
from keras.preprocessing.image import ImageDataGenerator, array_to_img,img_to_array,load_img
from keras.layers import Conv2D,Lambda,Flatten,Activation
from keras.models import load_model
from keras import Sequential
from keras.layers import Conv2D,Lambda,Flatten,Activation, Input
from keras.optimizers import Adam
import os
import pickle
import itertools
test_datagen = ImageDataGenerator(rescale=1. / 255)
model=load_model('first_try_maxout.h5',custom_objects={'tf':tf})
model2 = Model(inputs=model.input,outputs=model.get_layer('maxout4').output)



/usr/local/lib/python2.7/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
import itertools
def decode_batch(out):
    ret = []
    for j in range(out.shape[0]):
        out_best = list(np.argmax(out[j, 2:], 1))
        out_best = [k for k, g in itertools.groupby(out_best)]
        outstr = []
        for c in out_best:
           
                outstr.append(c)
        ret.append(outstr)
    return ret

In [4]:
def sliding_windows(img):
   
    arr=[]
    for i in range(32):
        s=4*i
        arr.append(img[:,s:s+32,:])
    print len(arr)
    a = np.asarray(arr)
    out=model2.predict(a,batch_size=32,verbose=1)
    print type(out)
    return np.reshape(out,(32,128))

In [5]:
with open("sequences.txt", "r") as f:
    seq_labels = pickle.load(f)


In [6]:
img = cv2.imread("../data/Seq_data/" + "Sequence_" + "90005" + ".png")
img = img / 255.
img = cv2.resize(img,(156,32))

labels_true=seq_labels["Sequence_" + "90005" + ".png"]

In [7]:

import matplotlib.pyplot as plt

plt.imshow(img)

In [13]:
seq=sliding_windows(img)

32
32/32 [==============================] - 0s 146us/step
<type 'numpy.ndarray'>


In [14]:
seq=np.expand_dims(seq,axis=0)
S=model4.predict(seq)

In [15]:
decode_batch(S)

[[56, 72, 63, 72, 35, 72, 66, 72, 70]]

In [16]:
print labels_true

[56, 63, 35, 66, 70]


In [67]:
def ctc_lambda_func(args):
    y_pred, labels, input_length, label_length = args

    # the 2 is critical here since the first couple outputs of the RNN
    # tend to be garbage:
    y_pred = y_pred[:, 2:, :]
    return K.ctc_batch_cost(labels, y_pred, input_length, label_length)


In [10]:
model3=load_model('first_try_LSTM_CTC.h5', custom_objects = {'<lambda>': lambda y_true, y_pred: y_pred})

Tensor("softmax/truediv:0", shape=(?, 32, 73), dtype=float32)


In [11]:
model4 = Model(inputs=model3.get_layer('the_input').input,outputs=model3.get_layer('softmax').output)